In [63]:
!pip install transformers huggingface-hub cloudinary pillow requests h5py tqdm numpy torch open-clip-torch

In [1]:
import argparse
import os
import sys
from pathlib import Path
from io import BytesIO
import json

import numpy as np
import hnswlib
import requests
import time
import h5py
from PIL import Image, ImageFile  # Import thêm ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True  # Cho phép đọc ảnh bị lỗi nhẹ
from tqdm import tqdm

# Check dependencies
try:
    import torch
    import open_clip
except ImportError as e:
    print(f"❌ Missing dependency: {e}")
    print("\nPlease install required packages:")
    print("  conda activate hnsw-backend-venv")
    print("  pip install open-clip-torch")
    sys.exit(1)

# Check dependencies
try:
    import cloudinary
    import cloudinary.api
    import cloudinary.uploader
    from transformers import AutoModel, AutoProcessor
    import torch
except ImportError as e:
    print(f"❌ Missing dependency: {e}")
    print("\nPlease install required packages:")
    print("  pip install transformers huggingface-hub cloudinary pillow requests h5py tqdm numpy torch")
    sys.exit(1)

In [2]:
def load_biomedclip():
    """Load BiomedCLIP model using open_clip"""
    print("\n🤖 Loading BiomedCLIP model...")
    print("   Model: hf-hub:microsoft/BiomedCLIP-PubMedBERT_256-vit_base_patch16_224")
    print("   This may take a few minutes on first run (downloads ~2GB)\n")
    
    # Load BiomedCLIP using open_clip
    model, preprocess_train, preprocess_val = open_clip.create_model_and_transforms(
        'hf-hub:microsoft/BiomedCLIP-PubMedBERT_256-vit_base_patch16_224'
    )
    tokenizer = open_clip.get_tokenizer('hf-hub:microsoft/BiomedCLIP-PubMedBERT_256-vit_base_patch16_224')
    
    # Move to GPU if available
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.to(device)
    model.eval()  # Set to evaluation mode
    
    print(f"✅ Model loaded successfully!")
    print(f"   Device: {device}")
    print(f"   Embedding dimension: 512")
    
    # Test the model
    test_text = "femur fracture"
    text_tokens = tokenizer([test_text]).to(device)
    with torch.no_grad():
        text_features = model.encode_text(text_tokens)
        text_features = text_features / text_features.norm(dim=-1, keepdim=True)
    
    print(f"\n🧪 Test encoding: '{test_text}'")
    print(f"   Output shape: {text_features.shape}")
    print(f"   ✅ Model is working correctly!")
    
    return model, preprocess_val, tokenizer, device

def encode_image(image_path, model, preprocess, device):
    """Encode a single image to embedding"""
    try:
        # Open and convert to RGB
        image = Image.open(image_path).convert('RGB')
        
        # Preprocess image
        image_tensor = preprocess(image).unsqueeze(0).to(device)
        
        # Generate embedding
        with torch.no_grad():
            image_features = model.encode_image(image_tensor)
            image_features = image_features / image_features.norm(dim=-1, keepdim=True)
        
        return image_features.cpu().numpy().astype(np.float32)[0]
        
    except Exception as e:
        print(f"\n❌ Failed to process {image_path}: {e}")
        return None
    
def get_text_embedding(text_input, model, tokenizer, device):
    """
    Chuyển đổi text (hoặc list các text) thành vector embedding đã được chuẩn hóa.
    
    Args:
        text_input: Một chuỗi (str) hoặc một danh sách chuỗi (list of str).
        model: Model BiomedCLIP đã load.
        tokenizer: Tokenizer của BiomedCLIP.
        device: 'cuda' hoặc 'cpu'.
        
    Returns:
        numpy.ndarray: Mảng vector embedding (shape: [n, 512]).
    """
    
    # 1. Xử lý đầu vào: Đảm bảo luôn là list để tokenizer hoạt động đúng
    if isinstance(text_input, str):
        text_input = [text_input]
        
    # 2. Tokenize: Chuyển text thành token ID
    # context_length của BiomedCLIP này là 256
    tokens = tokenizer(text_input).to(device)
    
    # 3. Inference: Chạy model để lấy feature
    with torch.no_grad():
        text_features = model.encode_text(tokens)
        
        # 4. Normalization (QUAN TRỌNG): 
        # Cần chia cho độ dài vector (L2 norm) để đưa về đơn vị chuẩn.
        # Nếu không làm bước này, tính Cosine Similarity sẽ bị sai.
        text_features = text_features / text_features.norm(dim=-1, keepdim=True)
        
    # 5. Chuyển về Numpy array để dễ lưu vào HNSW/Database
    return text_features.cpu().numpy()

In [3]:
model, processor, tokenizer, device = load_biomedclip()


🤖 Loading BiomedCLIP model...
   Model: hf-hub:microsoft/BiomedCLIP-PubMedBERT_256-vit_base_patch16_224
   This may take a few minutes on first run (downloads ~2GB)

✅ Model loaded successfully!
   Device: cuda
   Embedding dimension: 512

🧪 Test encoding: 'femur fracture'
   Output shape: torch.Size([1, 512])
   ✅ Model is working correctly!


In [4]:
dataset = json.load(open('cloudinary_urls.json'))

In [5]:
print(len(dataset))

3366


In [6]:
urls = []
all_embeddings_numpy = []

for i in tqdm(range(len(dataset))):
    url = dataset[i]['url']
    image_path = dataset[i]['local_path']
    embeddings = encode_image(image_path, model, processor, device)
    if embeddings is not None:
        urls.append(url)
        all_embeddings_numpy.append(embeddings)

all_embeddings = np.array(all_embeddings_numpy)
norms = np.linalg.norm(all_embeddings_numpy, axis=1, keepdims=True)
all_embeddings_numpy = all_embeddings_numpy / norms

100%|██████████| 3366/3366 [01:00<00:00, 55.85it/s]


In [7]:
output_path = "../backend/temp/Medical_Embedded.h5"

with h5py.File(output_path, "w") as outfile:
    outfile.create_dataset("urls", data=np.array(urls, dtype='S')) 
    outfile.create_dataset("embeddings", data=all_embeddings_numpy)

In [9]:
output_bin_path = "../backend/temp/Medical_Embedded.bin"

data = h5py.File(output_path, "r")
p = hnswlib.Index(space='cosine', dim=512)
p.init_index(max_elements=10000, ef_construction=400, M=200)
p.add_items(data["embeddings"])
p.set_ef(400)


In [10]:
p.save_index(output_bin_path)

In [11]:
xyz = p.knn_query(get_text_embedding("arm", model, tokenizer, device), 10)
print(xyz)

(array([[1147, 3107,   54, 1993, 1793, 1741,  885, 2999, 3269,  616]],
      dtype=uint64), array([[0.5979632 , 0.60067916, 0.6017135 , 0.6060069 , 0.6070913 ,
        0.60986036, 0.61100316, 0.6114818 , 0.61228967, 0.61520696]],
      dtype=float32))


In [12]:
print(dataset[2999]["url"])

https://res.cloudinary.com/dp4qen6gz/image/upload/v1765208122/medical/fractures_2/IMG0002064.jpg
